<a href="https://colab.research.google.com/github/Sourav-Manik/AI-Assignments/blob/main/Assignment_8_Audio_Classification_(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Audio classification using CNN

In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import keras
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt

Now convert the audio data files into PNG format images or basically extracting the Spectrogram for every Audio. (Below code is useful for CNN as we are converting data into image file)

In [ ]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
# Audio_Labels = 'fold1 fold2'.split()

Audio_Labels = ['air conditioner' ,'car horn' ,'children playing', 'dog bark', 'drilling', 'engine idling', 'gunshot', 'jackhammer','siren','street music']

# Audio_Labels.split()

for g in Audio_Labels:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'/content/drive/MyDrive/AI/Audio_Classification_Dataset/Audio_Labels/{g}'):
        songname = f'/content/drive/MyDrive/AI/Audio_Classification_Dataset/Audio_Labels/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 576x576 with 0 Axes>

In [ ]:
!pip install split_folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders

In [ ]:
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio('./img_data/', output="./data", seed=1337, ratio=(.8, .2)) # default values

Copying files: 240 files [00:00, 4154.27 files/s]


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rescale=1./255, # rescale all pixel values from 0-255, so aftre this step all our pixel values are in range (0,1)
        shear_range=0.2, #to apply some random tranfromations
        zoom_range=0.2, #to apply zoom
        horizontal_flip=True) # image will be flipper horiztest_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = datagen.flow_from_directory(
        './data/train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        shuffle = False)

test_set = datagen.flow_from_directory(
        './data/val',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical',
        shuffle = False )

Found 190 images belonging to 10 classes.
Found 50 images belonging to 10 classes.


In [ ]:
def CNN_Model(): 
  global model 
  model = Sequential()
  input_shape=(64, 64, 3)#1st hidden layer
  model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
  model.add(AveragePooling2D((2, 2), strides=(2,2)))
  model.add(Activation('relu'))#2nd hidden layer
  model.add(Conv2D(64, (3, 3), padding="same"))
  model.add(AveragePooling2D((2, 2), strides=(2,2)))
  model.add(Activation('relu'))#3rd hidden layer
  model.add(Conv2D(64, (3, 3), padding="same"))
  model.add(AveragePooling2D((2, 2), strides=(2,2)))
  model.add(Activation('relu'))#Flatten
  model.add(Flatten())
  model.add(Dropout(rate=0.5))#Add fully connected layer.
  model.add(Dense(64))
  model.add(Activation('relu'))
  model.add(Dropout(rate=0.5))#Output layer
  model.add(Dense(10))
  model.add(Activation('softmax'))

  epochs = 50
  batch_size = 4
  learning_rate = 0.01
  decay_rate = learning_rate / epochs
  momentum = 0.9
  sgd = tf.keras.optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
  model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

  return CNN_Model


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 31, 32)        896       
                                                                 
 average_pooling2d (AverageP  (None, 15, 15, 32)       0         
 ooling2D)                                                       
                                                                 
 activation (Activation)     (None, 15, 15, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 64)        18496     
                                                                 
 average_pooling2d_1 (Averag  (None, 7, 7, 64)         0         
 ePooling2D)                                                     
                                                                 
 activation_1 (Activation)   (None, 7, 7, 64)          0

In [ ]:
model.fit_generator(
        training_set,
        steps_per_epoch= 6//4, #len(train_set)//batch_size
        epochs=50,
        validation_data=test_set,
        validation_steps=200)

Epoch 1/50
1/1 [==============================] - ETA: 0s - loss: 2.3071 - accuracy: 0.0938

1/1 [==============================] - 2s 2s/step - loss: 2.3071 - accuracy: 0.0938 - val_loss: 2.2988 - val_accuracy: 0.1600
Epoch 2/50
1/1 [==============================] - 0s 368ms/step - loss: 2.2512 - accuracy: 0.1562
Epoch 3/50
1/1 [==============================] - 0s 366ms/step - loss: 2.3402 - accuracy: 0.1562
Epoch 4/50
1/1 [==============================] - 0s 358ms/step - loss: 2.3239 - accuracy: 0.0667
Epoch 5/50
1/1 [==============================] - 0s 352ms/step - loss: 2.3764 - accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 343ms/step - loss: 2.3125 - accuracy: 0.0312
Epoch 7/50
1/1 [==============================] - 0s 374ms/step - loss: 2.2551 - accuracy: 0.2500
Epoch 8/50
1/1 [==============================] - 0s 349ms/step - loss: 2.3012 - accuracy: 0.0312
Epoch 9/50
1/1 [==============================] - 0s 357ms/step - loss: 2.2502 - accuracy: 0.2500
Epoch 10/50
1/1 [==============================] - 0s 351ms/step - loss: 2.2992 - accu

In [ ]:
#Model Evaluation
model.evaluate_generator(generator=test_set, steps=50)#OUTPUT

[2.299072504043579, 0.20000000298023224]

End